# Line chart (benchmark analysis)

In [ ]:



gapminder = pd.read_csv('uni.csv',
                        header=0,
                        usecols=["University Name", "Arts and humanities ranking THE2024", "Engineering & Technology ranking THE24", "Natural science rankings QS24", "Life science/ medical rankings QS24", "Economics & Business Rankings THE24"])

@capture("graph")
def barchart(data_frame, x, y, title=None, text=None):
    fig_bar = px.bar(data_frame=data_frame, x=x, y=y, title=title, text=text)
    
    fig_bar.update_layout(barmode='stack', xaxis={'categoryorder':'total ascending'})
    fig_bar.update_layout(
    title=go.layout.Title(
    text="<sup>Smaller number means higher national ranking</sup>",
    xref="paper",
    x=0
        ),)

    return fig_bar


def create_subject():
    """Function returns a page to perform analysis on university level."""

    page_subject = vm.Page(
        title="Uni rankings by subject",
        description="Discovering how different NZ universities are ranked through the years",
        layout=vm.Layout( grid=[[1, 1, 1, 1, 1]] + [[0, 0, 0, 0, 0]] * 4,
            row_min_height="100px",
            row_gap="24px",),
        components=[
            vm.Graph(
                id="barchart",
                figure=barchart(
                    gapminder,
                    x="University Name",
                    y="Arts and humanities ranking THE2024",
                    text="Arts and humanities ranking THE2024"
                ),
            ),
            vm.Card(
            text="""
               scroll down! 
               ### National rankings of universities based on subject field
               
                 - Subject fields to choose from: 
                    - 🏛️ Arts and humanities: Art, History, Philosophy, Literature, Classics, Media Studies, Film studies 
                    - ⚙️ Engineering & Technology: Electronic, Computer engineering, Biomedical engineering, Computer Science, IT, Food Technology
                    - 🔬 Natural science: Physics, Chemistry, Biology 
                    - 🩺 Life science/ medical: Pharmaceuticals, Biotechnology, Biomedical Science 
                    - 🏭 Economics & Business: Economics, Marketing, Finance
                    
                >
                >  Choose subject field in selector on your left. 
                >
             
  
            
            """,
        ),
        ],
        controls=[
            vm.Parameter(
                targets=["barchart.y"],
                selector=vm.Dropdown(
                    options=["Arts and humanities ranking THE2024", "Engineering & Technology ranking THE24", "Natural science rankings QS24", "Life science/ medical rankings QS24", "Economics & Business Rankings THE24"], multi=False, value="Arts and humanities ranking THE2024", title="Choose subject field"
                ),
            ),
             vm.Parameter(
                targets=["barchart.text"],
                selector=vm.Dropdown(
                    options=["Arts and humanities ranking THE2024", "Engineering & Technology ranking THE24", "Natural science rankings QS24", "Life science/ medical rankings QS24", "Economics & Business Rankings THE24"], multi=False, value="Arts and humanities ranking THE2024", title="Choose subject field to see specific rankings on the bars"
                ),
            ),
        ],
    )

    return page_subject


IS_JUPYTERLAB = 'true'







In [56]:


                returns a page to show scores of different metrics on each university."""
    page_years = vm.Page(
        title="Universities' Scores in metrics",
        description="Discovering how different NZ universities are scored with different metrics",
        layout=vm.Layout(grid=[[0],]),
        components=[
            vm.Tabs(
                tabs=[
                    vm.Container(
                        title="University of Auckland",
                        layout=vm.Layout(grid=[[1, 0, 0]]),
                        components=[
                            vm.Graph(
                                id="uoa",
                                figure=radarchart(
                                    gapminder_1,
                                    r="r",
                                    theta="theta",
                                    title="University of Auckland",
                                    markers=True,
                                    hover_name="theta",
                                    line_close=True
                                ),
                            ),
                            vm.Card(
                                text="""
                                ![](assets/images/uoa2.png#my-image2)
                            
                                
                                The University of Auckland is New Zealand’s leading university, located in the country’s largest city. 
                                
                                It offers a comprehensive range of programs and is known for its strong emphasis on research and innovation. 

                            
                                         
                                > ### Highlighting the Strengths (Have a quick read)

                                > - **Academic Reputation (82.2)**: UOA is globally recognized for its academic excellence, 
                                         meaning you'll be learning from some of the best minds in the world.

                                > - **Employment Outcomes (92.9)**: With such a high score, UOA graduates are in hot demand by employers. 
                                        This translates to strong job prospects and exciting career opportunities right after graduation.

                                > - **Research Quality (88.3)**: Dive into groundbreaking research at UOA, 
                                         where innovative projects and discoveries are a daily norm. You'll be part of a university that pushes the boundaries of knowledge.

                                > - **International Research Network (87.4)**: Collaborate with top researchers worldwide.
                                         UOA's strong international connections mean you'll gain a global perspective and invaluable experience.  

                                

                                Choosing the University of Auckland means opting for a university that's highly respected, offers excellent job outcomes, and excels in research. 
                                It's a place where you'll thrive academically and professionally, all while staying close to the vibrant life of Auckland. 🌟📚🌍

                                """,
                            ),
                        ],
                    ),
                    vm.Container(
                        title="University of Otago",
                        layout=vm.Layout(grid=[[1, 0, 0]]),
                        components=[
                            vm.Graph(
                                id="otago",
                                figure=radarchart(
                                    gapminder_1,
                                    r="r_otago",
                                    theta="theta",
                                    title="University of Otago",
                                    markers=True,
                                    hover_name="theta",
                                    line_close=True
                                ),
                            ),
                            vm.Card(
                                text="""
                                # ![](assets/images/otago.png)
                                
                                The University of Otago is New Zealand’s oldest university, located in the scenic city of Dunedin. 
                                
                                It offers a rich variety of programs and is renowned for its vibrant campus life and strong research focus.
                                
                                > ### Highlighting the Strengths (Have a quick read)
                                
                                > - **Employment Outcomes (59.1)**: Otago graduates are highly sought after by employers, leading to strong job prospects and successful career paths right after graduation.
                                
                                > - **Research Quality (75.8)**: Engage in high-impact research at Otago, where students and faculty work on innovative projects that drive forward scientific and academic knowledge.
                                
                                > - **International Research Network (79.5)**: Otago boasts strong international collaborations, giving students the opportunity to participate in global research initiatives and broaden their academic horizons.
                                
                                Choosing the University of Otago means joining a university that values academic excellence, offers significant research opportunities, and fosters a supportive and dynamic student community. It's a place where you'll not only grow academically but also enjoy a vibrant and fulfilling university experience in a beautiful setting. 🌟📚🌏

                                """,
                            ),
                        ],
                    ),
                    vm.Container(
                        title="Massey University",
                        layout=vm.Layout(grid=[[1, 0, 0]]),
                        components=[
                            vm.Graph(
                                id="massey",
                                figure=radarchart(
                                    gapminder_1,
                                    r="r_massey",
                                    theta="theta",
                                    title="Massey University",
                                    markers=True,
                                    hover_name="theta",
                                    line_close=True
                                ),
                            ),
                            vm.Card(
                                text="""
                                # ![](assets/images/massey.png)
                                
                               ### Massey University
                                Massey University is renowned for its innovative teaching methods and extensive research programs, with campuses in Palmerston North, Albany, and Wellington.
                                
                                It offers a wide range of programs and emphasizes practical, hands-on learning experiences.
                                
                                > ### Highlighting the Strengths (Have a quick read)
                                
                                > - **International Research Network (85.1)**: Massey has an extensive international research network, allowing students to collaborate on global projects and gain international experience.
                                
                                > - **Research Quality (60.5)**: Engage in high-quality research at Massey, where innovative projects and discoveries are highly encouraged.
                                
                                > - **Employment Outcomes (49.2)**: Massey graduates are well-prepared for the workforce, with solid job prospects after graduation.
                                
                                Choosing Massey University means embracing innovation and benefiting from a global perspective, with numerous opportunities for international collaboration. 🌟🌐📚

                                """,
                            ),
                        ],
                    ),
                    vm.Container(
                        title="Victoria University of Wellington",
                        layout=vm.Layout(grid=[[1, 0, 0]]),
                        components=[
                            vm.Graph(
                                id="victoria",
                                figure=radarchart(
                                    gapminder_1,
                                    r="r_victoria",
                                    theta="theta",
                                    title="Victoria University of Wellington",
                                    markers=True,
                                    hover_name="theta",
                                    line_close=True
                                ),
                            ),
                            vm.Card(
                                text="""
                                # ![](assets/images/victoria.png)
                                
                                ### Victoria University of Wellington
                                Victoria University of Wellington is celebrated for its strong emphasis on research and academic excellence. Located in New Zealand's capital, it offers unique opportunities for engagement with government and industry.
                                
                                It offers a diverse range of programs and is known for its vibrant campus life and strong community engagement.
                                
                                > ### Highlighting the Strengths (Have a quick read)
                                
                                > - **Employment Outcomes (71.6)**: Graduates from Victoria have strong employment prospects, with many finding successful careers shortly after graduation.
                                
                                > - **Research Quality (69.4)**: Victoria excels in research, providing students with opportunities to participate in cutting-edge projects.
                                
                                > - **International Research Network (70.7)**: Victoria's strong international connections mean students can participate in global research initiatives and gain a global perspective.
                                
                                Choosing Victoria University of Wellington means gaining access to excellent academic programs and strong career opportunities in the heart of New Zealand's capital. 🌟🏙️📚
                                """,
                            ),
                        ],
                    ),
                    vm.Container(
                        title="University of Waikato",
                        layout=vm.Layout(grid=[[1, 0, 0]]),
                        components=[
                            vm.Graph(
                                id="waikato",
                                figure=radarchart(
                                    gapminder_1,
                                    r="r_waikato",
                                    theta="theta",
                                    title="University of Waikato",
                                    markers=True,
                                    hover_name="theta",
                                    line_close=True
                                ),
                            ),
                            vm.Card(
                                text="""
                                # ![](assets/images/waikato.png)
                                
                                ### University of Waikato
                                The University of Waikato, located in Hamilton, is known for its innovative approach to education and strong ties to the community.
                                
                                It offers a diverse range of programs and emphasizes research excellence and practical learning.
                                
                                > ### Highlighting the Strengths (Have a quick read)
                                
                                > - **Research Quality (74.3)**: Waikato excels in research, providing students with opportunities to participate in cutting-edge projects and innovations.
                                
                                > - **International Research Network (55.1)**: The university's global connections mean you'll have access to a diverse range of research opportunities and collaborations.
                                
                                Choosing the University of Waikato means joining a university that values research and practical experience. It's a place where you can engage in high-quality research while being part of a supportive and innovative community. 🌟🔬📚

                                """,
                            ),
                        ],
                    ),
                    vm.Container(
                        title="University of Canterbury",
                        layout=vm.Layout(grid=[[1, 0, 0]]),
                        components=[
                            vm.Graph(
                                id="canterbury",
                                figure=radarchart(
                                    gapminder_1,
                                    r="r_canterbury",
                                    theta="theta",
                                    title="University of Canterbury",
                                    markers=True,
                                    hover_name="theta",
                                    line_close=True
                                ),
                            ),
                            vm.Card(
                                text="""
                                # ![](assets/images/canterbury.png)
                                
                               ### University of Canterbury
                                The University of Canterbury, located in Christchurch, is renowned for its strong engineering and science programs.
                                
                                It offers a wide range of academic programs and is committed to providing a high-quality education through research and practical learning.
                                
                                > ### Highlighting the Strengths (Have a quick read)
                                
                                > - **Employment Outcomes (82.3)**: Canterbury graduates are highly sought after by employers, ensuring strong job prospects and exciting career opportunities.
                                
                                > - **International Research Network (63)**: The university's international collaborations provide students with opportunities to engage in global research projects.
                                
                                Choosing the University of Canterbury means being part of a respected institution with excellent job prospects and strong international research connections. It's a place where you can thrive academically and professionally. 🌟🔧📚


                                """,
                            ),
                        ],
                    ),
                    vm.Container(
                        title="Lincoln University",
                        layout=vm.Layout(grid=[[1, 0, 0]]),
                        components=[
                            vm.Graph(
                                id="lincoln",
                                figure=radarchart(
                                    gapminder_1,
                                    r="r_lincoln",
                                    theta="theta",
                                    title="Lincoln University",
                                    markers=True,
                                    hover_name="theta",
                                    line_close=True
                                ),
                            ),
                            vm.Card(
                                text="""
                                # ![](assets/images/lincoln.png)
                                
                                ### Lincoln University
                                Lincoln University, located near Christchurch, is a specialist land-based university with a strong focus on agriculture and environmental science.
                                
                                It offers unique programs tailored to these fields and emphasizes research and practical learning.
                                
                                > ### Highlighting the Strengths (Have a quick read)
                                
                                > - **Research Quality (68.4)**: Lincoln excels in research, particularly in areas related to agriculture and environmental science, providing students with opportunities to contribute to significant discoveries.
                                
                                Choosing Lincoln University means gaining expertise in specialized fields with strong research support. It's a place where you can engage in high-quality research while being close to nature. 🌟🌱📚


                                """,
                            ),
                        ],
                    ),
                    vm.Container(
                        title="AUT",
                        layout=vm.Layout(grid=[[1, 0, 0, 0]]),
                        components=[
                            vm.Graph(
                                id="aut",
                                figure=radarchart(
                                    gapminder_1,
                                    r="r_aut",
                                    theta="theta",
                                    title="AUT",
                                    markers=True,
                                    hover_name="theta",
                                    line_close=True
                                ),
                            ),
                            vm.Card(
                                text="""
                                # ![](assets/images/aut.png)
                                
                                ### Auckland University of Technology (AUT)
                                Auckland University of Technology (AUT), located in Auckland, is known for its innovative teaching methods and strong industry connections.
                                
                                It offers a dynamic learning environment with a focus on practical skills and real-world experience.
                                
                                > ### Highlighting the Strengths (Have a quick read)
                                
                                > - **Research Quality (84)**: AUT's commitment to research excellence provides students with opportunities to engage in high-impact projects and innovations.
                                
                                > - **International Research Network (52.6)**: AUT's strong international connections mean you'll have access to a global perspective and valuable experience.
                                
                                Choosing AUT means embracing innovation and benefiting from strong industry ties. It's a place where you can receive a forward-thinking education and gain practical skills for your future career. 🌟💡📚






                                """,
                            ),
                        ],
                    ),
                ],
            )
        ]
    )
    return page_years

# dashboard = vm.Dashboard(
#     pages=[
#         create_metrics()
#     ],
#     navigation=vm.Navigation(
#         nav_selector=vm.NavBar(
#             items=[
#                 vm.NavLink(
#                     label="Rankings of subject field",
#                     pages=["Universities' Scores in metrics"],
#                     icon="Auto Stories",
#                 ),
#             ]
#         ),
#     ),
# )

# if not IS_JUPYTERLAB:
#     app = Vizro().build(dashboard)
#     server = app.dash.server
    
#     if __name__ == "__main__":  
#         app.run(port=8080)
# else:
#     Vizro(assets_folder="assets").build(dashboard).run(port=8082)


In [11]:
import pandas as pd
import vizro.models as vm
import vizro.plotly.express as px
from vizro import Vizro
from vizro.tables import dash_ag_grid
from vizro.actions import export_data, filter_interaction
import plotly.graph_objects as go
from vizro.models.types import capture


Vizro._reset()

IS_JUPYTERLAB = 'true'


gapminder = pd.read_csv('uni.csv',
                        header=0,
                        usecols=["University Name", "Arts and humanities ranking THE2024", "Engineering & Technology ranking THE24", "Natural science rankings QS24", "Life science/ medical rankings QS24", "Economics & Business Rankings THE24"])

@capture("graph")
def barchart(data_frame, x, y, title=None, text=None, template=None):
    fig_bar = px.bar(data_frame=data_frame, x=x, y=y, title=title, text=text, template=template)
    
    fig_bar.update_layout(barmode='stack', xaxis={'categoryorder':'total ascending'})
    fig_bar.update_layout(
    title=go.layout.Title(
    text="<sup>Smaller number means higher national ranking</sup>",
    xref="paper",
    x=0
        ),)

    return fig_bar


def create_subject():
    """Function returns a page to perform analysis on university level."""

    page_subject = vm.Page(
        title="Uni rankings by subject",
        description="Discovering how different NZ universities are ranked through the years",
        layout=vm.Layout( grid=[[1, 1, 1, 1, 1]] + [[0, 0, 0, 0, 0]] * 4,
            row_min_height="100px",
            row_gap="24px",),
        components=[
            vm.Graph(
                id="barchart",
                figure=barchart(
                    gapminder,
                    x="University Name",
                    y="Arts and humanities ranking THE2024",
                    text="Arts and humanities ranking THE2024",
                    template="ygridoff"
                ),
            ),
            vm.Card(
            text="""
               scroll down! 
               ### National rankings of universities based on subject field
               
                #### - Subject fields to choose from: 
                
                 
                > - 🏛️ Arts and humanities: Art, History, Philosophy, Literature, Classics, Media Studies, Film studies 
                    
                > - ⚙️ Engineering & Technology: Electronic, Computer engineering, Biomedical engineering, Computer Science, IT, Food Technology
                    
                > - 🔬 Natural science: Physics, Chemistry, Biology 
                    
                > - 🩺 Life science/ medical: Pharmaceuticals, Biotechnology, Biomedical Science 
                    
                > - 🏭 Economics & Business: Economics, Marketing, Finance
                
                    
                >
                >  Choose subject field in selector on your left. 
                >
             
  
            
            """,
        ),
        ],
        controls=[
            vm.Parameter(
                targets=["barchart.y"],
                selector=vm.Dropdown(
                    options=["Arts and humanities ranking THE2024", "Engineering & Technology ranking THE24", "Natural science rankings QS24", "Life science/ medical rankings QS24", "Economics & Business Rankings THE24"], multi=False, value="Arts and humanities ranking THE2024", title="Choose subject field"
                ),
            ),
             vm.Parameter(
                targets=["barchart.text"],
                selector=vm.Dropdown(
                    options=["Arts and humanities ranking THE2024", "Engineering & Technology ranking THE24", "Natural science rankings QS24", "Life science/ medical rankings QS24", "Economics & Business Rankings THE24"], multi=False, value="Arts and humanities ranking THE2024", title="Choose subject field to see specific rankings on the bars"
                ),
            ),
        ],
    )

    return page_subject

dashboard = vm.Dashboard(
    pages=[
        create_subject()
    ],
    navigation=vm.Navigation(
        nav_selector=vm.NavBar(
            items=[
                vm.NavLink(
                    label="Rankings of subject field",
                    pages=["Uni rankings by subject"],
                    icon="Auto Stories",
                ),
            ]
        ),
    ),
)

if not IS_JUPYTERLAB:
    app = Vizro().build(dashboard)
    server = app.dash.server
    
    if __name__ == "__main__":  
        app.run(port=8080)
else:
    Vizro(assets_folder="assets").build(dashboard).run(port=8082)
